<a href="https://colab.research.google.com/github/sobti/TSAI/blob/master/S11/resnet_model_album_cam__ocp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#Mounting Google Colab and importing generic Packages
import torchvision
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from albumentations.pytorch import ToTensor
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from google.colab import drive
import os
import sys
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import torchvision
import torch
import albumentations
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,PadIfNeeded,
    Transpose, RandomCrop ,ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose,Cutout)

drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/Colab Notebooks\resnet')
os.chdir('/content/gdrive/My Drive/Colab Notebooks/resnet')


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
#importing self-created packages:
import get_max_lr as t                                                         #import train - test module .This is class to train and evaluate the model
import load as ld                                                               #dataloader
import mod_summ as m                                                            #Getting model summary package
import resnet_s11 as bb                                                         #Getting Classes for Resnet and Basic Block
import s11_album as al                                                          #Getting albumentation module
import override_album as oa                                                     # overriding the albumentation class
import GradCm as gd                                                             #Grad Cam Module
import plot_cam   as pt                                                         #Plotting Image with Gradcam
import lr_linear  as lr                                                         #exponential LR update


In [15]:
net=bb.Net().to('cuda') 
m.summ_print(net,(3,32,32))# Model summary.

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,728
       BatchNorm2d-5          [-1, 128, 32, 32]             256
              ReLU-6          [-1, 128, 32, 32]               0
         MaxPool2d-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,456
              ReLU-9          [-1, 128, 16, 16]               0
      BatchNorm2d-10          [-1, 128, 16, 16]             256
           Conv2d-11          [-1, 128, 16, 16]         147,456
             ReLU-12          [-1, 128, 16, 16]               0
      BatchNorm2d-13          [-1, 128, 16, 16]             256
           Conv2d-14          [-1, 256,

In [0]:
train_transforms,test_transforms=al.album_transform(albumentations)             #Albumentation class 

In [17]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=None)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=None)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
# Initialize the dataset, pass the augmentation pipeline as an argument to init function
train_ds = oa.PyTorchImageDataset(image=trainset, augmentations=train_transforms)
test_ds =  oa.PyTorchImageDataset(image=testset, augmentations=test_transforms)

In [0]:
batch_size=512
trainloader,testloader=ld.load_test_train(batch_size,train_ds,test_ds,torchvision,torch)

In [0]:
train_losses = []
test_losses = []
train_acc = []
test_acc = []
train_acc_graph=[]
history = {"lr": []}
getlr=0.001
optimizer = optim.SGD(net.parameters(),lr=getlr)                               # start LR is 0.00001
lr_schedule = lr.LinearLR(optimizer)
criterion=nn.CrossEntropyLoss()
best_loss = None
device='cuda'
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_acc1=t.train(net, device, trainloader, optimizer,train_losses,train_acc,tqdm,criterion)
    train_acc_graph.append(train_acc1)
    lr_schedule.step()
    lrn=lr_schedule.get_lr()[0]
    history["lr"].append(lrn)

  0%|          | 0/98 [00:00<?, ?it/s]

EPOCH: 0


Loss=1.6675653457641602 Batch_id=97 Accuracy=32.54: 100%|██████████| 98/98 [00:21<00:00,  4.64it/s]
  0%|          | 0/98 [00:00<?, ?it/s]

EPOCH: 1


Loss=1.3560529947280884 Batch_id=97 Accuracy=45.72: 100%|██████████| 98/98 [00:20<00:00,  4.67it/s]
  0%|          | 0/98 [00:00<?, ?it/s]

...

Loss=1.7668672800064087 Batch_id=97 Accuracy=45.34: 100%|██████████| 98/98 [00:20<00:00,  4.82it/s]
  0%|          | 0/98 [00:00<?, ?it/s]

EPOCH: 13


Loss=2.336355447769165 Batch_id=97 Accuracy=31.82: 100%|██████████| 98/98 [00:20<00:00,  4.79it/s]
  0%|          | 0/98 [00:00<?, ?it/s]

EPOCH: 14


Loss=2.834559440612793 Batch_id=11 Accuracy=34.59:  12%|█▏        | 12/98 [00:02<00:18,  4.66it/s]

In [12]:
max_lr1=history['lr'][train_acc_graph.index(max(train_acc_graph))]
plt.plot(history['lr'],train_acc_graph)
net1=bb.Net().to('cuda')                                                        # Creating new model for final training.

ValueError: ignored

In [11]:
import train_test_model_resnet18_missclass as t1                                   #import train - test module .This is class to train and evaluate the model
train_losses = []
test_acc = []
test_losses = []
train_acc = []
train_loss_graph=[]
train_acc_graph=[]
test_acc_graph=[]
test_loss_graph=[]
EPOCHS = 24
optimizer = optim.SGD(net.parameters(),lr=0.001,weight_decay=0.0020)
criterion=nn.CrossEntropyLoss()
scheduler=torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr,steps_per_epoch=len(trainloader),pct_start=0.208,epochs=24, anneal_strategy=None,div_factor=10)

device='cuda'
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_loss,train_acc1=t1.train(net, device, trainloader, optimizer, epoch,train_losses,train_acc,tqdm,criterion)
    train_loss_graph.append(train_loss)
    train_acc_graph.append(train_acc1)
    test_loss,test_acc1=t1.test(net, device, testloader,test_losses,test_acc,tqdm,criterion,torch)
    test_loss_graph.append(test_loss)    
    test_acc_graph.append(test_acc1)
    scheduler.step(test_loss)
    

TypeError: ignored

In [0]:
plt.plot(test_acc_graph)                
plt.plot(train_acc_graph)
plt.plot(test_loss_graph)
plt.plot(train_loss_graph)       
plt.title("Model Graph")          
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['test','train','test_loss','train_loss'] ,loc='upper left')
plt.show()  
plt.savefig('Upload_graph.png')           

In [0]:
cam2=[]
data2=[]
k=[]
dat=[]
number=0
p=[]
l=[]
data2=[]
for im in range(25):
  
  dat1,label = next(iter(testloader))
  dat1=dat1.to('cuda')
  output=net(dat1)
  pred = output.argmax(dim=1, keepdim=True)
  
  for i in range(label.size()[0]):
   if pred[i].item() != label[i].item():
    p.append(pred[i].item()) 
    l.append(label[i].item()) 
    label1=label[i]
    data1=dat1[i]
    data2.append(dat1[i].to('cpu'))
    label1=label1.unsqueeze(0) 
    Gd1=gd.GradCm()
    cam1=Gd1.img(data1,label1,net)
    cam2.append(cam1) 
    number = len(cam2)
    if number >= 25:
        print(number)
        break
  else:
   continue
  break 

In [0]:
plott=pt.Plot()
#l=l.to('cpu')
#data2=data2.to('cpu')
plott.fig(data2,cam2,l,p)


fgfghfhfgfhhggdrfrrfdsvfgdffdv          